In [60]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos, exp, log,e

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [61]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def df_da(a):
    return -3 * a**2 - 0.5 * a**(-0.5)


def df_db(b):
    return 3 * cos(3 * b) + 2.5 * b ** 1.5

def df_dc(c):
    return 1/(c**2)

def gradf(a, b, c):

  
  
  return [df_da(a), df_db(b), df_dc(c)] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [62]:
# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

# -----------
numerical_grad = [0, 0, 0] # TODO
# -----------

h = 0.000001


numerical_grad[0]= (f(2+h,3,4)-f(2,3,4))/h
numerical_grad[1]= (f(2,3+h,4)-f(2,3,4))/h
numerical_grad[2]= (f(2,3,4+h)-f(2,3,4))/h





for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [1]:
from math import exp, log

class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other): # exactly as in the video
        print(self, other , "self and the other ")
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
    
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
    
        return out
    def __radd__(self, other):
        return self + other 

    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward

        return out


    def __truediv__ (self, other):
        out = self * other**-1

        return out
    
    
    

    def exp(self):
        out = Value(exp(self.data), (self,),"exp")

        def _backward():
            self.grad += exp(self.data) * out.grad
            print(self.grad)

        out._backward = _backward
        return out

    def log(self):
        out = Value(log(self.data),(self,), 'log')

        def _backward():
            print(self, "log self")
            self.grad += 1/self.data * out.grad
            print(self.grad, "log grad")
       

        out._backward = _backward
        return out

    def __neg__(self):
        print(self, "negate self")
        negated_value = -self.data
        out = Value(negated_value, (self,), "-")
        def _backward():
            self.grad += -1 * out.grad
            print(self.grad, "negate grad")
        

        out._backward = _backward
        print(out.grad, "negate out grad")
        return out

    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        print(topo)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()


In [3]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)

loss = -(probs[3].log()) # dim 3 acts as the label for this input example


loss.backward()
print(loss.grad, "loss")
print(loss._prev, "prev")

print(logits[3].grad, "logits grad")


ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


Value(data=1.0) 0 self and the other 
Value(data=1.0) Value(data=20.085536923187668) self and the other 
Value(data=21.085536923187668) Value(data=0.1353352832366127) self and the other 
Value(data=21.220872206424282) Value(data=2.718281828459045) self and the other 
Value(data=-2.1755153626167147) negate self
0.0 negate out grad
[Value(data=1.0), Value(data=2.718281828459045), Value(data=-2.0), Value(data=0.1353352832366127), Value(data=0), Value(data=0.0), Value(data=1.0), Value(data=1.0), Value(data=3.0), Value(data=20.085536923187668), Value(data=21.085536923187668), Value(data=21.220872206424282), Value(data=23.939154034883327), Value(data=0.04177257051535045), Value(data=0.11354961935990122), Value(data=-2.1755153626167147), Value(data=2.1755153626167147)]
-1.0 negate grad
Value(data=0.11354961935990122) log self
-8.806722608469956 log grad
0.8390245074625319
0.041772570515350445
0.005653302662216329
-0.8864503806400986
1 loss
{Value(data=-2.1755153626167147)} prev
-0.88645038064